# Reccomendation of 5 Mac Miller songs!

### Importing libraries and setting up document

In [1]:
# setting up libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import gensim
import json
from pandasql import *
mysql = lambda q: sqldf(q, globals())
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from lyricsgenius import Genius # lyrics genius is a fantastic library that helps us take lyrics from the Genius API
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

//anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
#setting up Genius API
client_id = 'AqnRTduqC75eC1V8Fr3EKTnLep6I_-rk99_RdWPEpSlfeyg5IoZrLs9KRLQBK6Db'
token = 'r_Gny4srkI6fW0JKR5zYU8LofeURTtx4Up7kQa_kqXlkoy6O82vZ_UaUZTWuq8fg'
genius = Genius(token)

#Removes [chorus] [bridge] etc. tags.
genius.remove_section_headers = True

#excluding repeat songs that might hinder analysis
genius.excluded_terms = ["(Remix)", "(Live)"]
#artist = genius.search_artist("Kendrick Lamar", max_songs=10)

In [8]:
macmiller = pd.read_csv('table_w_topics.csv')
macmiller.head(5)

,Unnamed: 0,id,song.full_title,song.artist_names,song.lyrics,UniqueWordsPercent,AlbumName,topic1,topic2,topic3,topic4,topic5,topic6,topic7
0,0,0,Kickin' Incredibly Dope Shit (Intro) by Mac Mi...,Mac Miller,when youre young not much matter when you find...,0.509395,K.I.D.S. (2010),0.6104,0.0006,0.0006,0.0630,0.0006,0.3240,0.0006
1,1,1,Outside by Mac Miller,Mac Miller,do you mind if we smoke this blunt outside cau...,0.350649,K.I.D.S. (2010),0.0006,0.0006,0.0006,0.9967,0.0006,0.0006,0.0006
2,2,2,Get Em Up by Mac Miller,Mac Miller,if you in the ride turn the volume real high i...,0.360000,K.I.D.S. (2010),0.9964,0.0006,0.0006,0.0006,0.0006,0.0006,0.0006
3,3,3,Nikes on My Feet by Mac Miller,Mac Miller,woah haha yeah and the nike on my foot keep my...,0.424171,K.I.D.S. (2010),0.9961,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007
4,4,4,Senior Skip Day by Mac Miller,Mac Miller,kid kid yeah haha im just high why they hatin ...,0.463700,K.I.D.S. (2010),0.0007,0.0007,0.0007,0.6336,0.0007,0.3630,0.0007


### Setting up functions to perform LDA on song

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = nltk.corpus.stopwords.words('english')
more_stop_words = ["gon", "em", "yeah", "uh", "could", "ima", "me", "my", "and", "to",
                   "in", "hey", "like", "yeah", "im", "go", "know", "get", "shit"]

# Map POS tag to first character lemmatizing_and_stop() accepts
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Funtion to lemmatize words and remove stop words
def lemmatizing_and_stop(lyrics):
    prac = [lemmatizer.lemmatize(lyric, get_wordnet_pos(lyric)) for lyric in nltk.word_tokenize(lyrics)]
    return ' '.join([lyric for lyric in prac if lyric not in stop_words or more_stop_words])

# Lemmatizing lyrics and removing stop words from them
for i in range(len(all_albums)):
    for j in range(len(all_albums[i][["song.lyrics"]])):
        all_albums[i]["song.lyrics"] = all_albums[i]["song.lyrics"].apply(lambda lyric: lemmatizing_and_stop(lyric))
        